# Magnetic System Explorer

Comprehensive notebook for magnetic system analysis with real-time visualization.
Supports any magnetic material with interactive simulation and phase diagram generation.

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spinlab.analysis.visualization import SpinVisualizer
import time

# Configure matplotlib for notebook
%matplotlib inline

# Initialize visualizer
viz = SpinVisualizer()
print("Magnetic System Explorer Ready!")
print("Available materials: Fe, Ni, Co, Mn")

## System Configuration

Configure your magnetic system parameters here:

In [ ]:
# System parameters - MODIFY THESE FOR YOUR ANALYSIS
MATERIAL = 'Fe'           # Options: 'Fe', 'Ni', 'Co', 'Mn'
SYSTEM_SIZE = (16, 16, 1) # Grid dimensions (x, y, z)
EXCHANGE_J = -0.02        # Exchange coupling (negative = ferromagnetic)
MAGNETIC_FIELD = [0, 0, 0.1]  # Applied field [Bx, By, Bz] in Tesla

# Simulation parameters
TEMPERATURE = 100         # Temperature in Kelvin
MC_STEPS = 1000          # Monte Carlo steps
UPDATE_INTERVAL = 50     # Visualization update frequency

print(f"Configuration: {MATERIAL} system, size {SYSTEM_SIZE}")
print(f"Exchange J = {EXCHANGE_J} eV, T = {TEMPERATURE} K")

## Create Magnetic System

Build the magnetic system with specified parameters:

In [ ]:
# Create system builder
build_system = viz.create_system_builder()

# Build the magnetic system
system = build_system(
    material=MATERIAL, 
    size=SYSTEM_SIZE,
    J=EXCHANGE_J,
    B_field=MAGNETIC_FIELD
)

# System information
print(f"System created: {len(system.positions)} magnetic sites")
print(f"Initial energy: {system.total_energy():.4f} eV")
print(f"Initial magnetization: {np.linalg.norm(system.total_magnetization()):.3f}")

## Initial Spin Configuration

Visualize the random initial spin state:

In [ ]:
# Plot initial random configuration
viz.plot_spin_grid_skyrmion_style(
    system.spin_config,
    title=f"{MATERIAL} Initial Random Configuration",
    figsize=(8, 8)
)

## Real-Time Simulation

Run Monte Carlo simulation with live monitoring of energy, magnetization, and spin evolution:

In [ ]:
# Create real-time monitor
monitor = viz.create_live_monitor(system, figsize=(15, 5))

# Run simulation with live updates
print(f"Starting simulation: T={TEMPERATURE}K, {MC_STEPS} steps")
result = viz.run_live_simulation(
    system, 
    monitor, 
    temperature=TEMPERATURE,
    n_steps=MC_STEPS,
    update_every=UPDATE_INTERVAL
)

print(f"\nSimulation complete!")
print(f"Final energy: {result['final_energy']:.4f} eV")
print(f"Final |M|: {np.linalg.norm(result['final_magnetization']):.3f}")
print(f"Acceptance rate: {result['acceptance_rate']:.3f}")

## Equilibrated Configuration

Visualize the final equilibrated spin state:

In [ ]:
# Plot equilibrated configuration
viz.plot_spin_grid_skyrmion_style(
    system.spin_config,
    title=f"{MATERIAL} Equilibrated State (T={TEMPERATURE}K)",
    figsize=(8, 8)
)

# Analysis summary
final_energy = system.total_energy()
final_mag = system.total_magnetization()
print(f"\nEquilibrated System Analysis:")
print(f"Total energy: {final_energy:.4f} eV")
print(f"Energy per site: {final_energy/len(system.positions):.6f} eV")
print(f"Magnetization vector: [{final_mag[0]:.3f}, {final_mag[1]:.3f}, {final_mag[2]:.3f}]")
print(f"Magnetization magnitude: {np.linalg.norm(final_mag):.3f}")

## Temperature Sweep Analysis

Perform temperature sweep to find phase transitions and critical temperature:

In [ ]:
# Temperature sweep parameters
T_MIN = 10    # Minimum temperature (K)
T_MAX = 400   # Maximum temperature (K)
N_POINTS = 20 # Number of temperature points
STEPS_PER_T = 300  # MC steps per temperature

# Generate temperature range
T_range = np.linspace(T_MIN, T_MAX, N_POINTS)

print(f"Temperature sweep: {T_MIN}K to {T_MAX}K ({N_POINTS} points)")
print(f"Steps per temperature: {STEPS_PER_T}")

# Reset system for sweep
system.random_configuration()

# Run temperature sweep with live plotting
phase_results = viz.temperature_sweep_live(
    system, 
    T_range, 
    steps_per_T=STEPS_PER_T
)

print(f"\nTemperature sweep complete for {MATERIAL}!")

## Phase Transition Analysis

Analyze the temperature sweep results to identify phase transitions:

In [ ]:
# Extract data from phase results
temperatures = np.array(phase_results['T'])
energies = np.array(phase_results['E'])
magnetizations = np.array(phase_results['M'])

# Calculate heat capacity (numerical derivative of energy)
if len(temperatures) > 3:
    dE_dT = np.gradient(energies, temperatures)
    heat_capacity = -dE_dT
    
    # Find critical temperature (steepest drop in magnetization)
    dM_dT = np.gradient(magnetizations, temperatures)
    T_c_idx = np.argmin(dM_dT)
    T_c = temperatures[T_c_idx]
    
    # Phase transition analysis
    print(f"Phase Transition Analysis for {MATERIAL}:")
    print(f"Estimated critical temperature: {T_c:.1f} K")
    print(f"Low-T magnetization (T={temperatures[0]:.1f}K): {magnetizations[0]:.3f}")
    print(f"High-T magnetization (T={temperatures[-1]:.1f}K): {magnetizations[-1]:.3f}")
    print(f"Energy range: {energies.min():.4f} to {energies.max():.4f} eV/site")
    
    # Phase classification
    if magnetizations[0] > 0.5 and magnetizations[-1] < 0.2:
        print("Phase transition type: Ferromagnetic → Paramagnetic")
    elif magnetizations[0] < 0.2:
        print("System appears paramagnetic at all temperatures")
    else:
        print("Partial magnetic ordering observed")
        
    # Plot heat capacity
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(temperatures, heat_capacity, 'go-', markersize=4, linewidth=2)
    plt.axvline(T_c, color='red', linestyle='--', alpha=0.7, label=f'T_c ≈ {T_c:.1f}K')
    plt.xlabel('Temperature (K)')
    plt.ylabel('Heat Capacity')
    plt.title('Heat Capacity vs Temperature')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(temperatures, dM_dT, 'mo-', markersize=4, linewidth=2)
    plt.axvline(T_c, color='red', linestyle='--', alpha=0.7, label=f'T_c ≈ {T_c:.1f}K')
    plt.xlabel('Temperature (K)')
    plt.ylabel('dM/dT')
    plt.title('Magnetization Derivative')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("Need more temperature points for detailed analysis")

## Material Comparison

Compare different magnetic materials at the same temperature:

In [ ]:
# Materials to compare
materials = ['Fe', 'Ni', 'Co', 'Mn']
comparison_temp = 150  # K
comparison_steps = 500

print(f"Comparing materials at T={comparison_temp}K")

# Create comparison plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

material_data = {}

for i, material in enumerate(materials):
    print(f"Simulating {material}...")
    
    # Create system
    sys = build_system(
        material=material, 
        size=(12, 12, 1), 
        J=EXCHANGE_J
    )
    
    # Quick equilibration
    from spinlab.monte_carlo import MonteCarlo
    mc = MonteCarlo(sys, temperature=comparison_temp)
    result = mc.run(n_steps=comparison_steps, verbose=False)
    
    # Store results
    material_data[material] = {
        'energy': result['final_energy'],
        'magnetization': result['final_magnetization'],
        'spins': sys.spin_config.copy()
    }
    
    # Plot spin configuration
    plt.sca(axes[i])
    
    spins = sys.spin_config
    n_spins = len(spins)
    grid_size = int(np.sqrt(n_spins))
    
    # Create grid positions
    x_1d = np.arange(grid_size)
    y_1d = np.arange(grid_size)
    X, Y = np.meshgrid(x_1d, y_1d)
    x, y = X.flatten(), Y.flatten()
    
    # Spin components
    sx, sy, sz = spins[:, 0], spins[:, 1], spins[:, 2]
    r = np.sqrt(sx**2 + sy**2 + sz**2)
    background_colors = sz / r
    
    # Plot
    plt.scatter(x, y, c=background_colors, cmap='jet', s=20, vmin=-1, vmax=1)
    plt.quiver(x, y, sx, sy, color='black', scale=25, width=0.003)
    
    # Format
    energy_per_site = result['final_energy'] / len(sys.positions)
    mag_magnitude = np.linalg.norm(result['final_magnetization'])
    plt.title(f'{material} (T={comparison_temp}K)\nE={energy_per_site:.4f} eV/site, |M|={mag_magnitude:.3f}')
    plt.axis('equal')
    plt.xlim(-1, grid_size)
    plt.ylim(-1, grid_size)
    plt.xticks([])
    plt.yticks([])

plt.tight_layout()
plt.show()

# Summary table
print(f"\nMaterial Comparison Summary (T={comparison_temp}K):")
print("Material | Energy/site (eV) | |Magnetization|")
print("-" * 45)
for material in materials:
    data = material_data[material]
    energy_per_site = data['energy'] / (12*12)  # 12x12 grid
    mag = np.linalg.norm(data['magnetization'])
    print(f"{material:8s} | {energy_per_site:13.4f} | {mag:13.3f}")

## Performance Benchmarking

Test simulation speed and system capabilities:

In [ ]:
# Performance test
print("Performance Benchmark:")
print("=" * 40)

# Test different system sizes
test_sizes = [(10, 10, 1), (20, 20, 1), (30, 30, 1)]
test_steps = 100

for size in test_sizes:
    print(f"\nTesting {size[0]}x{size[1]} system ({size[0]*size[1]} sites):")
    
    # Create test system
    test_system = build_system('Fe', size=size, J=-0.02)
    
    # Time the simulation
    from spinlab.monte_carlo import MonteCarlo
    mc = MonteCarlo(test_system, temperature=100)
    
    start_time = time.time()
    result = mc.run(n_steps=test_steps, verbose=False)
    elapsed = time.time() - start_time
    
    # Calculate performance metrics
    speed = test_steps / elapsed
    time_per_sweep = elapsed / test_steps * 1000  # ms
    sites_per_second = len(test_system.positions) * speed
    
    print(f"  Speed: {speed:.1f} sweeps/s")
    print(f"  Time per sweep: {time_per_sweep:.2f} ms")
    print(f"  Sites per second: {sites_per_second:.0f}")
    
    # Estimate time for large simulations
    time_10k = 10000 / speed
    if time_10k < 60:
        print(f"  10k sweeps: {time_10k:.1f} seconds")
    elif time_10k < 3600:
        print(f"  10k sweeps: {time_10k/60:.1f} minutes")
    else:
        print(f"  10k sweeps: {time_10k/3600:.1f} hours")

# Check Numba acceleration
from spinlab.core.fast_ops import HAS_NUMBA
print(f"\nNumba acceleration: {'Active' if HAS_NUMBA else 'Not available'}")
if not HAS_NUMBA:
    print("Install numba for significant speedup: pip install numba")

## Analysis Summary and Export

Summary of the analysis and options to save results:

In [ ]:
# Create comprehensive analysis summary
print("=" * 60)
print(f"MAGNETIC SYSTEM ANALYSIS SUMMARY")
print("=" * 60)
print(f"Material: {MATERIAL}")
print(f"System size: {SYSTEM_SIZE} ({np.prod(SYSTEM_SIZE)} sites)")
print(f"Exchange coupling: {EXCHANGE_J} eV")
print(f"Applied field: {MAGNETIC_FIELD} T")
print()

if 'result' in locals():
    print(f"Simulation Results (T={TEMPERATURE}K):")
    print(f"  Final energy: {result['final_energy']:.4f} eV")
    print(f"  Final |M|: {np.linalg.norm(result['final_magnetization']):.3f}")
    print(f"  Acceptance rate: {result['acceptance_rate']:.3f}")
    print()

if 'T_c' in locals():
    print(f"Phase Transition Analysis:")
    print(f"  Critical temperature: {T_c:.1f} K")
    print(f"  Temperature range: {T_MIN}-{T_MAX} K")
    print(f"  Low-T magnetization: {magnetizations[0]:.3f}")
    print(f"  High-T magnetization: {magnetizations[-1]:.3f}")
    print()

print("Files generated:")
print("  - Spin configuration plots")
print("  - Temperature sweep data")
print("  - Phase transition analysis")
print("  - Material comparison")

# Option to save data
save_data = input("\nSave analysis data to files? (y/n): ")
if save_data.lower() == 'y':
    import os
    
    # Create results directory
    results_dir = f"{MATERIAL}_analysis_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save temperature sweep data
    if 'phase_results' in locals():
        np.savetxt(f"{results_dir}/temperature_sweep.txt", 
                   np.column_stack([temperatures, energies, magnetizations]),
                   header="Temperature(K) Energy(eV/site) Magnetization",
                   fmt="%.6f")
        print(f"Temperature sweep data saved to {results_dir}/temperature_sweep.txt")
    
    # Save final configuration
    np.save(f"{results_dir}/final_spin_config.npy", system.spin_config)
    print(f"Final spin configuration saved to {results_dir}/final_spin_config.npy")
    
    # Save summary
    with open(f"{results_dir}/analysis_summary.txt", 'w') as f:
        f.write(f"Magnetic System Analysis Summary\n")
        f.write(f"Material: {MATERIAL}\n")
        f.write(f"System size: {SYSTEM_SIZE}\n")
        f.write(f"Exchange coupling: {EXCHANGE_J} eV\n")
        f.write(f"Applied field: {MAGNETIC_FIELD} T\n")
        if 'T_c' in locals():
            f.write(f"Critical temperature: {T_c:.1f} K\n")
    
    print(f"Analysis summary saved to {results_dir}/analysis_summary.txt")
    print(f"All results saved in directory: {results_dir}/")

print("\nAnalysis complete!")